In [2]:
!pip install unidecode
!pip install transformers==4.2.2
!pip install pyreadr
!pip install unidecode
!pip install ftfy
!pip install git+https://github.com/legalnlp21/legalnlp

     |████████████████████████████████| 241 kB 7.2 MB/s 
     |████████████████████████████████| 1.8 MB 8.1 MB/s 
     |████████████████████████████████| 895 kB 56.6 MB/s 
     |████████████████████████████████| 2.9 MB 25.2 MB/s 
     |████████████████████████████████| 409 kB 8.2 MB/s 
     |████████████████████████████████| 64 kB 2.7 MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41934 sha256=e3124daba954a1dadf7f171848aea2b459e736efe38ab1830a5a872828468f22
  Stored in directory: /root/.cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
Successfully built ftfy
  Cloning https://github.com/legalnlp21/legalnlp to /tmp/pip-req-build-4octg4hl
  Running command git clone -q https://github.com/legalnlp21/legalnlp /tmp/pip-req-build-4octg4hl
  Created wheel for legalnlp: filename=legalnlp-1.0.0-py3-none-any.whl size=4242 sha256=b86bb7d3fe631def05100b026bc80d9acf483bbd11c9a4e5d2b90b06563ae93d
  Stored in directory: /tmp/pip-ephem-wheel-cache

In [3]:
from IPython.display import Image
from IPython.display import clear_output
import pickle
import random
import unidecode
import re
import ftfy
import os
import copy
import torch
import torch.nn as nn
import torch.utils.data as tdata
import torch.optim as optim
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig
from transformers import BertForPreTraining, BertModel, BertTokenizer, BertForMaskedLM, BertForNextSentencePrediction, BertForQuestionAnswering
# manipulação numérica e de dataframes
import numpy as np
import pandas as pd
# gráficos e ajustes visuais
import matplotlib.pyplot as plt
import textwrap
from tqdm import tqdm

In [6]:
# Import functions of legalnlp
from legalnlp.get_premodel import *
from legalnlp.clean_functions import *

In [10]:
#Downloading BERTikal
get_premodel('bert')

In [9]:
%%time
data=pd.read_csv('https://raw.githubusercontent.com/legalnlp21/legalnlp/main/demo/data_base.csv')
data.drop(columns=['Unnamed: 0'],inplace=True)
data['text'] = data['text'].apply(lambda x:clean_bert(x))
model = '/content/BERTikal/BERTikal'
tokenizer= '/content/BERTikal/BERTikal/vocab.txt'
data

CPU times: user 395 ms, sys: 733 µs, total: 395 ms
Wall time: 455 ms


In [ ]:
def extract_features_bert(path_model, path_tokenizer, data,gpu=True):
    if gpu:
        device = torch.device('cuda')
        os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
        bert_tokenizer = BertTokenizer.from_pretrained(path_tokenizer, do_lower_case=False)
        bert_model = BertModel.from_pretrained(path_model).to(device)
        bert = data.apply(lambda x: bert_tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))
        data_text = list(data)
        encoded_inputs = bert_tokenizer(data_text, padding=True, truncation=True, max_length=512, return_tensors="pt")
        input_ids = encoded_inputs['input_ids'].to(device)
        clear_output()
        # Criando o nosso vetor de features
        features = []
    
        # Aplicando o modelo pré-treinado em cada frase e adicionando-o ao nosso vetor
    
        for i in tqdm(range(len(data)), mininterval=40, maxinterval=500):
    
            with torch.no_grad():
    
                last_hidden_states = bert_model(
                    input_ids[i:(i+1)])[1].cpu().numpy().reshape(-1).tolist()
            features.append(last_hidden_states)
    else:
        bert_tokenizer = BertTokenizer.from_pretrained(path_tokenizer, do_lower_case=False)
        bert_model = BertModel.from_pretrained(path_model)
        bert = data.apply(lambda x: bert_tokenizer.encode(
            x, add_special_tokens=True, max_length=512, truncation=True))
        encoded_inputs = bert_tokenizer(
            data_text, padding=True, truncation=True, max_length=512, return_tensors="pt")
        input_ids = encoded_inputs['input_ids']
        # Criando o nosso vetor de features
        features = []
    
        # Aplicando o modelo pré-treinado em cada frase e adicionando-o ao nosso vetor
    
        for i in tqdm(range(len(data))):
    
            with torch.no_grad():
    
                last_hidden_states = bert_model(
                    input_ids[i:(i+1)])[1].cpu().numpy().reshape(-1).tolist()
            features.append(last_hidden_states)
    features = np.array(features)
    features[:2]
    df_features = pd.DataFrame(features)
    return df_features



    # os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
    #device = torch.device('cuda:0')
features= extract_features_bert(model,tokenizer,data['text'])

 96%|█████████▌| 4884/5093 [03:20<00:08, 24.35it/s]

In [ ]:
features